In [ ]:
import os
import numpy
import random
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [ ]:
## 訓練、測試資料集、輸出位置
TRAIN_IMAGES_ROOT_PATH = "mnist_png/training/"
TEST_IMAGES_ROOT_PATH = "mnist_png/testing/"

In [ ]:
## 全部類別
all_classes = os.listdir(TRAIN_IMAGES_ROOT_PATH)
all_classes

In [ ]:
## 讀取所有類別的圖片
train_images = []
train_classes = []
test_images = []
test_classes = []

for classes in all_classes:
    train_image_array = os.listdir(TRAIN_IMAGES_ROOT_PATH+classes) ##取得在訓練資料中,每個類別裡的圖片
    test_image_array = os.listdir(TEST_IMAGES_ROOT_PATH+classes) ##取得在測試資料中，每個類別裡的圖片
    print("正在讀取類別"+classes+"的圖片")
    for image_name in train_image_array:
        image_path = TRAIN_IMAGES_ROOT_PATH+classes+"/"+image_name 
        image = load_img(image_path,target_size=(28,28)) ## 讀取圖片
        image = img_to_array(image) ## 圖片轉換成像素點Array
        train_images.append(image)
        train_classes.append(classes)
    for image_name in test_image_array:
        image_path = TEST_IMAGES_ROOT_PATH+classes+"/"+image_name
        image = load_img(image_path,target_size=(28,28,3)) ##讀取圖片
        image = img_to_array(image)
        test_classes.append(classes)
        test_images.append(image)

In [ ]:
test_data_length = len(test_classes)

In [ ]:
# keras 輸入要求 numpy array
X = numpy.array(train_images,dtype="float") / 255.0 #把每個像素點縮至 0 ~ 1
Y = numpy.array(train_classes)
Y = to_categorical(Y) ## 類似one-hot-endcoding,類別 A =[1,0,0],類別 B =[0,1,0],類別 C =[0,0,1]
val_X = numpy.array(test_images,dtype="float") / 255.0
val_Y = numpy.array(test_classes)
val_Y = to_categorical(val_Y)

In [ ]:
X

In [ ]:
Y

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),padding="same",input_shape=(28,28,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(5,5),padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=len(all_classes),activation="softmax"))
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:
history = model.fit(X,Y,validation_data=(val_X,val_Y),shuffle=True,epochs=5,batch_size=256,verbose=1)

In [ ]:
print(history.history.keys())

In [ ]:
pyplot.plot(history.history['acc'])
pyplot.plot(history.history['val_acc'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='lower right')
pyplot.show()

In [ ]:
## 檢查是否Overfitting
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper right')
pyplot.show()
# 當兩者越來越分開，則代表有overfitting發生

In [ ]:
result = model.predict(val_X)

In [ ]:
prediction = numpy.argmax(result,axis=1)

In [ ]:
prediction

In [ ]:
pyplot.figure(figsize=(20,5))
pyplot.plot(result)
pyplot.ylabel('Predict')
pyplot.xlabel('Data')
pyplot.legend(all_classes, loc='upper left')
pyplot.show()

In [ ]:
fig , axes=pyplot.subplots(1,5,figsize=(10,10))
for i in range(5):
    index = random.randint(0,len(test_images)-1) 
    axes[i].set_title('Prediction : '+str(prediction[index]))
    axes[i].imshow(test_images[index].astype('uint8'))
    axes[i].set_xticks([])
    axes[i].set_yticks([])
pyplot.show()